# Smart bath


## Введение 
В данном проекте рассматривается пример реализации системы для автоматического поддержания температуры воды в бане. Система включает реле, которое автоматически включает нагреватель и подогревает воду до заданной температуры. После достижения нужной температуры система будет поддерживать её на оптимальном уровне, не допуская отклонения за заданные пределы.


## Участвующие устройства 
В проекте будут участвовать: 
1. Arduino Nano 
2. DS18D20 датчик температуры 
3. Реле, нагревающее воду 
4. Светодиод, для сигнализирование состояния реле (включен когда идет нагрев, выключен когда температура стабилизировалась) 
5. OLED дисплей (SSD1306), для вывода текущей температуры воды. 
6. 16-ти битный таймер, для замера времени «простоя» реле.



## Подключение всех устройств к Arduino:

1.	DS18B20 (датчик температуры):
* DQ (Data): Подключить к цифровому пину 2 на Arduino через подтягивающий резистор (4.7 кОм) между DQ и VCC.
* VCC: Подключить к 5V Arduino.
* GND: Подключить к GND Arduino.
2.	SSD1306 (OLED-дисплей):
* VCC: Подключить к 3.3V Arduino (или 5V, если ваш модуль поддерживает оба уровня).
* GND: Подключить к GND Arduino.
* SCL: Подключить к пину A5 (или SCL).
* SDA: Подключить к пину A4 (или SDA).
3.	Реле:
* IN: Подключить к цифровому пину 4 на Arduino.
* VCC: Подключить к 5V Arduino.
* GND: Подключить к GND Arduino.
4.	Светодиод:
* Аном (длинная ножка): Подключить через резистор (220 Ом) к цифровому пину 8 на Arduino.
* Катод (короткая ножка): Подключить к GND Arduino.



## Основная логика:

* DS18B20 считывает текущую температуру воды.
* Реле управляет нагревателем (например, ТЭН).
* OLED-дисплей отображает текущую температуру воды.
* Светодиод включается во время работы нагревателя, сигнализируя процесс нагрева.



## Код для DS18D20

```c
#include <OneWire.h>
#include <DallasTemperature.h>

#define ONE_WIRE_BUS 2 

OneWire oneWire(ONE_WIRE_BUS);
DallasTemperature DS18D20(&oneWire);

void setup() {
 Serial.begin(9600);
 oneWire.begin();

} 



void loop(){
 DS18D20.requestTemperatures();
 float currentTemp = DS18D20.getTempCByIndex(0);
 
 // Вместо Serial будет вывод на дисплей 
 Serial.print(“Текущая температура воды: ”);
 Serial.println(currentTemp);
}

## Код для таймера

Поскольку мы не хотим проверять температуру воды каждый такт и излишне загружать микроконтроллер, проверка температуры будет выполняться каждую минуту. Это достаточно для поддержания оптимальной температуры воды.

Для реализации этого мы настроим таймер, который будет генерировать прерывание каждую секунду. На каждое прерывание мы будем увеличивать счётчик секунд, а при достижении минуты счётчик сбросится, и произойдёт проверка температуры.

### Расчёты

Используем Timer1 (16-битный) с максимальным предделителем 1024:

* Тактовая частота:
$$ f_{CPU} = 16 , \text{МГц} $$
* Частота работы таймера:
$$ f_{timer} = \frac{16 , \text{МГц}}{1024} = 15625 , \text{Гц} $$
Для одного переполнения таймера потребуется:
$$ T_{overflow} = \frac{2^{16}}{f_{timer}} = \frac{65536}{15625} \approx 4.19 , \text{с} $$
Чтобы достичь 60 секунд, понадобится:
$$ \frac{60}{4.19} \approx 14.33 , \text{переполнений} $$
Однако работа с такими дробными значениями неудобна и затрудняет понимание кода. Поэтому вместо режима Normal (переполнений) мы выберем CTC-режим (Clear Timer on Compare Match). Это позволит настроить таймер так, чтобы он генерировал прерывание каждую секунду.

### Настройка CTC-режима

В режиме CTC мы можем установить значение для сравнения в регистре OCR1A, чтобы таймер сбрасывался при достижении этого значения:
$$ OCR1A = \frac{f_{timer}}{f_{прерываний}} - 1 = \frac{15625}{1} - 1 = 15624 $$

Каждую секунду будет происходить сравнение, генерироваться прерывание, и мы сможем обновить счётчик секунд.


```c

#include <avr/io.h>
#include <avr/interrupt.h>

volatile uint8_t seconds = 0;
float targetTemperature = 70.0;

void timer1_init() {
    // CTC Mode 
    TCCR1B |= (1 << WGM12); 


    // Включаем предделитель 1024
    TCCR1B |= (1 << CS12) | (1 << CS10); 

    // значение равное секунде 
    OCR1A = 15624; 

   
    // разрешаем прерывания 
    TIMSK1 |= (1 << OCIE1A); 
    sei();
}


void timer1_stop() {

  TIMSK1 &= ~(1 << OCIE1A);
  cli(); 

}




ISR(TIMER1_COMPA_vect) {
    seconds++;
    if (seconds >= 60) {
        seconds = 0;
  
       
        relayOn(targetTemperature);
    }
}




// подогрев воды пока вода не станет нужной температуры 

void relayOn(float mainTemperature){
 timer1_stop();
 
 PORTB ^= (1 << PORTB0); // включение светодиода


 // нагреваем воду, пока она не станет нужной нам температуры
}


void relayOff(){
 timer1_init();
 PORTB &= ~(1 << PORTB0); // выключение светодиода


 // выключение реле

} 


void setup {
    // настройка PB0
    pinMode(D8, OUTPUT);
}


## Код для SSD1306

```c

#include <Wire.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>

#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64
#define OLED_RESET    -1 

Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

void setup() {
  if (!display.begin(SSD1306_I2C_ADDRESS, 0x3C)) { // Адрес 0x3C
    Serial.println(F("SSD1306 не найден!"));
    for (;;);
  }
  display.clearDisplay();
  display.setTextSize(1);
  display.setTextColor(SSD1306_WHI
TE);
  display.setCursor(0, 0);
  display.println("Текущая температура воды:");
  display.display();
}

void loop() {
  // Оставляем дисплей включенным
}



## Итоговый код: 

```c

#include <avr/io.h>
#include <avr/interrupt.h>

#include <OneWire.h>
#include <DallasTemperature.h>

#include <Wire.h>
#include <Adafruit_GFX.h>
#include <Adafruit_SSD1306.h>

// Реле и цель температуры
#define RELAY_PIN 4
float targetTemperature = 70.0;

/*
 * Функционал DS18B20
 */
#define ONE_WIRE_BUS 2

OneWire oneWire(ONE_WIRE_BUS);
DallasTemperature DS18D20(&oneWire);

// Считываем текущую температуру
float getCurrentTemperature() {
    DS18D20.requestTemperatures();
    return DS18D20.getTempCByIndex(0);
}

/*
 * Функционал SSD1306
 */
#define SCREEN_WIDTH 128
#define SCREEN_HEIGHT 64
#define OLED_RESET -1

Adafruit_SSD1306 display(SCREEN_WIDTH, SCREEN_HEIGHT, &Wire, OLED_RESET);

// Инициализация дисплея
void displaySetUp() {
    if (!display.begin(SSD1306_I2C_ADDRESS, 0x3C)) {
        Serial.println(F("SSD1306 не найден!"));
        for (;;); // Бесконечный цикл, если дисплей не найден
    }
    display.clearDisplay();
    display.setTextSize(1);
    display.setTextColor(SSD1306_WHITE);
}

// Отображение температуры
void displayPrint(float temperature) {
    display.clearDisplay();
    display.setCursor(0, 0);
    display.print(F("Текущая температура: "));
    display.print(temperature);
    display.display();
}

/*
 * Функционал таймера
 */
volatile uint8_t seconds = 0;

void timer1_init() {
    // Режим CTC
    TCCR1B |= (1 << WGM12);

    // Предделитель 1024
    TCCR1B |= (1 << CS12) | (1 << CS10);

    // Значение для 1 секунды
    OCR1A = 15624;

    // Разрешаем прерывания
    TIMSK1 |= (1 << OCIE1A);
    sei(); // Глобальные прерывания
}

void timer1_stop() {
    TIMSK1 &= ~(1 << OCIE1A); // Отключаем прерывания таймера
    cli(); // Отключаем глобальные прерывания
}

// Прерывание таймера
ISR(TIMER1_COMPA_vect) {
    seconds++;
    if (seconds >= 60) {
        seconds = 0;
        relayOn(targetTemperature);
    }
}

/*
 * Управление реле
 */
void relayOn(float temperature) {
    timer1_stop();
    float currentTemp = getCurrentTemperature();
    displayPrint(currentTemp);

    if (currentTemp < temperature) {
        digitalWrite(RELAY_PIN, HIGH); // Включаем реле
        Serial.println(F("Включаю реле"));
    }
}

void relayOff() {
    digitalWrite(RELAY_PIN, LOW); // Выключаем реле
    Serial.println(F("Выключаю реле"));
    timer1_init(); // Запускаем таймер
}

/*
 * Нагрев воды до целевой температуры
 */
void heatWater() {
    float currentTemp = getCurrentTemperature();
    while (currentTemp < targetTemperature) {
        relayOn(targetTemperature);
        currentTemp = getCurrentTemperature();
    }
    relayOff();
}

/*
 * Основные функции Arduino
 */
void setup() {
    Serial.begin(9600);

    // Настраиваем реле и светодиод
    pinMode(RELAY_PIN, OUTPUT);
    digitalWrite(RELAY_PIN, LOW);

    // Инициализация дисплея и DS18B20
    displaySetUp();
    DS18D20.begin();

    // Инициализация таймера
    timer1_init();

    // Начальный нагрев воды
    heatWater();
}

void loop() {
    // Здесь можно добавить дополнительный функционал
}


## Завершение
Проект умной бани завершён. Все ключевые компоненты работают стабильно:
* Датчик температуры DS18B20 успешно считывает текущую температуру воды.
* Реле корректно управляет нагревателем, обеспечивая автоматический нагрев и поддержание оптимальной температуры.
* OLED-дисплей SSD1306 отображает текущую температуру воды, что делает систему удобной и информативной для пользователя.
* Таймер эффективно обрабатывает временные интервалы, минимизируя нагрузку на микроконтроллер.
* Светодиод визуально сигнализирует о работе нагревателя, добавляя простую, но полезную индикацию.

### Результат
Система готова к использованию в реальных условиях. Она демонстрирует, как можно сочетать аппаратное и программное обеспечение для автоматизации процессов в быту. Проект стал прекрасным примером создания компактного и функционального решения на базе Arduino Nano.

